In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import sqlite3
import datetime
import os
import time

start_time = datetime.datetime.now()
driver = webdriver.Chrome()
driver.get("http://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7&flag=0")
#driver.get("http://www.bseindia.com/markets/equity/EQReports/StockPrcHistori.aspx?expandable=7&scripcode=532532&flag=sp&Submit=G")

# Create Sqlite3 connection
conn = sqlite3.connect("scrapper_db")
cursor = conn.cursor()
print 'Time Taken:', datetime.datetime.now() - start_time

Time Taken: 0:01:20.506946


In [2]:
def move_to_db(stock_code):
    download_loc = "/home/himanshuchopra/Downloads/"
    data_path = download_loc + stock_code + ".csv"
    data_file = pd.read_csv(data_path)

    for value in data_file.iterrows():
        data_date = datetime.datetime.strptime(value[1][0],'%d-%B-%Y')
        #stock_code : Defined earlier
        open_price = value[1][1]
        high_price = value[1][2]
        low_price = value[1][3]
        close_price = value[1][4]
        wap = value[1][5]
        no_of_shares = value[1][6]
        no_of_trades = value[1][7]
        tot_turnover = value[1][8]
        delivb_qty = value[1][9]
        perct_deli_tra_qty = value[1][10]
        spread_hl = value[1][11]
        spread_co = value[1][12]
        n_created = datetime.datetime.now()
        to_db = (data_date, stock_code, open_price, high_price, low_price, close_price, wap, no_of_shares, no_of_trades,
                 tot_turnover, delivb_qty, perct_deli_tra_qty, spread_hl, spread_co, n_created)
        cursor.execute(("INSERT INTO stocks_data_int VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);"), to_db)
    conn.commit()
    
def delete_file(stock_code):
    try:
        os.remove("/home/himanshuchopra/Downloads/" + stock_code + ".csv")
    except:
        pass

In [ ]:
query = """select sc.stock_code, (case when scr.last_data_date is null then '2016-01-01' else last_data_date end) as min_date
            from stocks_config sc
            left join stocks_config_runlog scr on sc.stock_code = scr.stock_code and sc.exchange = scr.exchange;"""
stocks = pd.read_sql(query, con=conn)
i = 0
for key, value in stocks.iterrows():
    
    start_time = datetime.datetime.now()
    #Search stock based on stock id
    stock_element = driver.find_element_by_id('ctl00_ContentPlaceHolder1_GetQuote1_smartSearch')
    stock_element.clear()
    for i in range(0,len(value['stock_code'])):      
        stock_element.send_keys(value['stock_code'][i])
        time.sleep(1)
    stock_element.send_keys(Keys.RETURN)
    stock_element.send_keys(Keys.RETURN)   
    
    #Input From Date
    from_date_element = driver.find_element_by_id("ctl00_ContentPlaceHolder1_txtFromDate")
    from_date_element.clear()
    from_date_value = datetime.datetime.strptime(value['min_date'], '%Y-%m-%d').strftime('%d/%m/%Y')
    from_date_element.send_keys(from_date_value)
    #Input to Date
    to_date_element = driver.find_element_by_id("ctl00_ContentPlaceHolder1_txtToDate")
    to_date_element.click()
    calendar_element = driver.find_element_by_class_name("current")
    calendar_element.click()
    #Click on Submit
    submit_element = driver.find_element_by_id("ctl00_ContentPlaceHolder1_btnSubmit")
    submit_element.click()
    #Click on Download
    download_element = driver.find_element_by_id("ctl00_ContentPlaceHolder1_btnDownload1")
    download_element.send_keys(Keys.RETURN)
    time.sleep(4)
    #Get data into database
    move_to_db(value['stock_code'])
    #Delete File
    delete_file(value['stock_code'])
    print 'Time Taken for', value['stock_code'], datetime.datetime.now() - start_time
    if i == 10:
        break
    else:
        i = i + 1

In [4]:
# Delete duplicate data from stocks_data
query = """delete from stocks_data
where exists(select sd.data_date, sd.stock_code 
                from stocks_data sd 
                join stocks_data_int on stocks_data_int.data_date = stocks_data.data_date and 
                                        stocks_data_int.stock_code = stocks_data.stock_code);"""
cursor.execute(query)

# Insert data in stocks_data
query = """insert into stocks_data
select * from stocks_data_int;"""
cursor.execute(query)

# Truncate stocks_data_int
query = """delete from stocks_data_int;"""
cursor.execute(query)
conn.commit()

# Update stocks_config_runlog

query = """insert into stocks_config_runlog
select sc.stock_code, sc.exchange, date(min_date), date(max_date), last_run, 'himanshu.chopra'
from stocks_config sc
join (select stock_code, min(data_date) as min_date, max(data_date) as max_date, max(n_created) as last_run
from stocks_data
group by 1) sd on sc.stock_code = sd.stock_code;"""
cursor.execute(query)
conn.commit()